# **Projeto 1 introdução a redes neurais**

------------------------------------------------------------------------------------------------------------

## Objetivo: 

Implementar uma rede MLP, em python, sem usar pacotes prontos (e.g., Pytorch, Tensorflow, etc.)
Com a rede implementada, desenvolver dois modelos: um para classificação e um para regressão.
Avaliar os hiperparâmetros dos modelos variando o número de camadas, número de neurônio e taxas (eta e momentum).

## Importanto as bibliotecas necessárias

In [1]:
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt

## Funções de ativação 

A função linear retorna o próprio x

In [2]:
def linear(x): 
    return x 

A função degrau acima retorna 1 se x > 0 ou 0 caso contrário

In [3]:
def degrau(x): 
    return np.where(x >= 0, 1, 0) 

A função sigmoid logística retorna 
$$
f(x) = \frac{1}{1 + \exp(-x)}


In [4]:
def sigmoid(x): 
    return 1 / (1 + np.exp(-x)) 

A derivada da função sigmoid retorna 

$$
f'(x) = f(x) \cdot (1 - f(x))


In [ ]:
def sigmoid_derivada(x): 
    return x * (1 - x)

A função tangente hiperbólica retorna a tangente hiperbólica do x

In [5]:
def tanh(x):
    return np.tanh(x)

A função relu retorna o máximo entre o X e 0

In [6]:
def relu(x): 
    return np.maximum(0, x)

A função softmax converte a saída padrão em uma distribuição de probabilidade, da forma: 
$$
f(y) = \frac{\exp(y)}{\sum_{k} \exp(y_k)}

In [7]:
def softmax(x): 
    exp = np.exp(x - np.max(x))
    return exp / np.sum(exp, axis=-1, keepdims=True)

## Retropropagação

A retropropagação surgiu para resolver o problema de como atualizar os neurônios das camadas ocultas em redes neurais. Em redes profundas, esses neurônios não têm erros diretamente observáveis como na camada de saída. O objetivo da retropropagação é calcular esses erros e ajustar os pesos das camadas ocultas utilizando o método de gradiente descendente, permitindo que redes com múltiplas camadas aprendam de forma eficiente. As contas da retropropagação envolvem o cálculo do gradiente da função de erro em relação aos pesos, ajustando-os de forma iterativa para minimizar a diferença entre as saídas desejadas e as reais. Esses cálculos incluem a atualização dos pesos na camada de saída, com base no erro direto, e nas camadas ocultas, com base nos erros propagados pela rede, até a camada de entrada. O processo é feito usando o gradiente local, o qual depende da função de ativação de cada neurônio, garantindo que o erro seja distribuído corretamente entre as camadas da rede.

### A fórmula para a atualização dos pesos de um neurônio, é dada por:
$$
\Delta w_{kj} = \eta \cdot \delta_k \cdot x_j
$$

- Onde: 
    - $\eta$ é a taxa de aprendizado
    - $\delta_k$ é o gradiente local do neurônio $k$
    - $x_j$ é a saída do neurônio anterior $j$ (entrada para o neurônio $k$)

### O cálculo do gradiente local ($\delta_k$) é dado por dois casos: 

#### Neurônio de saída: 
$
\delta_k = e_k \cdot f'(v_k)
$
- Onde:
  - $\delta_k$: Gradiente local do neurônio $k$
  - $e_k$: Erro no neurônio de saída
  - $f'(v_k)$: Derivada da função de ativação com relação ao estado interno $v_k$


#### Caso 2: Neurônio oculto: 
$
\delta_j = f'(v_j) \cdot \sum_k \delta_k \cdot w_{kj}
$

- Onde:
  - $\delta_j$: Gradiente local do neurônio $j$ da camada oculta
  - $f'(v_j)$: Derivada da função de ativação com relação ao estado interno $v_j$
  - $\sum_k$: Soma sobre todos os neurônios $k$ da camada seguinte
  - $\delta_k$: Gradiente local do neurônio $k$ da camada seguinte
  - $w_{kj}$: Peso entre o neurônio $j$ da camada oculta e o neurônio $k$ da camada seguinte



### A fórmula de erro quadrático médio ou MSE, é dada por: 

$$
E(n) = \frac{1}{2} \sum_{k} \left( d_k(n) - y_k(n) \right)^2
$$

Onde: 
- $E(n)$ é a função de erro na época $n$.
- $d_k(n)$ é o valor desejado para o neurônio $k$ na época $n$.
- $y_k(n)$ é o valor atual da saída do neurônio $k$ na época $n$.
- A soma é realizada sobre todos os neurônios $k$ da camada de saída.

### Feedfoward e Feedback 


O algoritmo de retropropagação opera em duas fases principais. Na fase de feedforward, os dados de entrada percorrem a rede camada por camada até a saída, produzindo as previsões. Já na fase de feedback, os erros calculados na saída são propagados de volta, ajustando os pesos camada a camada. Esse processo possibilita calcular o gradiente dos neurônios ocultos e atualizar seus pesos

### Pseudocódigo 

- **Inicialização**: Configure os hiperparâmetros (taxa de aprendizado, número máximo de épocas, etc.) e inicialize os pesos da rede.
- **Treinamento**: Repita os passos abaixo até que o erro seja menor ou igual à tolerância definida ou até atingir o número máximo de épocas:
   - Aplique um padrão de entrada $x_i$ e o respectivo vetor de saída desejado $d_i$.
   - Realize a **propagação do sinal** (*feedforward*) da entrada até a saída.
   - Execute a **retropropagação dos erros** da saída para as camadas anteriores.
   - Atualize os pesos da rede com base nos gradientes calculados.
   - Retorne ao passo inicial para o próximo padrão de entrada.

### Implementação 

In [15]:
# Inicializando o momentum acumulado  
momentum_acumulado = None 

def retropropagação(X, y, pesos, bias, taxa_de_aprendizado, momentum): 
    """
    Função de retropropagação com feedforward e backpropagation
    
    Parâmetros:
    X: matriz de entradas (tamanho: número de amostras x número de características)
    y: vetor de saídas (tamanho: número de amostras)
    pesos: pesos da rede (incluindo pesos das camadas)
    bias: valores de bias das camadas
    taxa_aprendizado: taxa de aprendizado (eta)
    momentum: fator de momentum
    
    Retorna:
    pesos atualizados e bias ajustado
    """
    global momentum_acumulado

    # Inicializando o momentum acumulado se necessário (apenas na primeira execução)

    if momentum_acumulado is None: 
        momentum_acumulado = [np.zeros_like(peso) for peso in pesos] # Inicializa o momentum com o mesmo formato dos pesos

    #----------------------------------------------------------Feedforward---------------------------------------------------------------------- 
    camada_de_entrada = X # Dados de entrada X
    ativações = [X] # Lista que armazena as ativações, começando com a entrada
    lista_peso = [] # lista que armazena o peso de cada camada


    for peso, bias in zip(pesos, bias): 
        w = np.dot(camada_de_entrada, peso) + bias # Calcula o w da camada
        lista_peso.append(w) # Armazena na lista de pesos
        camada_de_entrada = sigmoid(w) #Aplicando a função de ativação sigmoid  
        ativações.append(camada_de_entrada) # Armazenando a ativação da camada 

    #----------------------------------------------------------Backpropagation------------------------------------------------------------------
    # Calculo do erro na camada de saída 
    gradiente = (ativações[-1] - y) * sigmoid_derivada(ativações[-1]) # Erro na camada de saída
    gradiente_w = [np.dot(ativações[-2].T, gradiente)] # Gradiente dos pesos da camada de saída
    gradiente_bias = [np.sum(gradiente, axis=0, keepdims=True)] # Gradiente dos bias da camada de saída

    # Calculando os gradientes para as camadas ocultas
    for camada in range(2, len(pesos) + 1):  
        w = lista_peso[-camada] # Obtém os pesos da camada atual na ordem inversa (da saída para a entrada)
        sig = sigmoid_derivada(sigmoid(w)) # Derivada da funcao de ativação sigmoid
        gradiente = np.dot(gradiente, pesos[-camada + 1].T) * sig # Propagando o erro
        gradiente_w.insert(0, np.dot(ativações[-camada - 1].T, gradiente)) # Gradiente dos pesos 
        gradiente_bias.insert(0, np.sum(gradiente, axis=0, keepdims=True)) # Gradiente dos vieses

    # Atualizando pesos e bias com momentum
    for i in range(len(pesos)): 
        # Atualiza o momentum acumulado
        momentum_acumulado[i] = momentum * momentum_acumulado[i] + gradiente_w[i] # Acumula o momentum
        pesos[i] -= taxa_de_aprendizado * momentum_acumulado[i] # Atualiza os pesos com o momentum acumulado
        bias[i] -= taxa_de_aprendizado * gradiente_bias[i] # Atualiza o bias

    return pesos, bias # Retorna os pesos e o bias ajustado

## Inicialização dos pesos

In [ ]:
def inicializar_pesos(tamanho_camadas): 
    """
    Inicializa os pesos e os bias de uma rede neural   
    
    Parâmetros: 
    tamanho_camadas: contém o número de neurônios em cada camada da rede
    
    Retorna: 
    pesos: lista de matrizes de pesos entre cada camada
    bias: lista de vetores de bias para cada camada
    """
    # Listas para armazenar os pesos e bias de cada camada
    pesos = []
    bias = []

    # Itera sobre cada par de camadas consecutivas (entrada e saída)
    for i in range(1, len(tamanho_camadas)):
        camada_entrada = tamanho_camadas[i - 1]
        camada_saida = tamanho_camadas[i]



